In [ ]:
import warnings
warnings.filterwarnings('ignore')

Viewing solar data with SunPy
---
In the first half of this tutorial, we'll look at how to search for, download, and plot remote sensing data using SunPy.

Searching and Downloading Data with SunPy
---
In order to search for some data, we have to select a time range and an instrument to search for. In addition, here we also specify the wavelength of interest.

In [ ]:
from sunpy.net import Fido, attrs as a
import astropy.units as u

Searching for data
---
Using the above defined search parameters, Fido can be used to search for data. For more information on searching for and downloading data see https://docs.sunpy.org/en/stable/guide/acquiring_data/index.html

In [ ]:
result = Fido.search(a.Time('2018/10/31 04:00', '2018/11/01 04:01'),
                     a.Instrument('AIA'),
                     a.Wavelength(19.3 * u.nm),
                     a.Sample(6*u.hour))
result

Downloading data
---
The results from a search can also be downloaded.

``Fido.fetch`` returns a list of the local location of the downloaded files.

In [ ]:
downloaded_files = Fido.fetch(result)
print(downloaded_files)

SunPy `Map` and `MapSequence`
---
Now we have downloaded some data, we can load it and plot it. ``sunpy.map.Map`` can be used to load any ``.fits`` file, creating a ``Map`` object. We can then take a look at the image stored by calling ``map.peek()``.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
import sunpy.map

## `MapSequence`

In [ ]:
map_seq = sunpy.map.Map(downloaded_files, sequence=True)

In [ ]:
map_seq.peek()

In [ ]:
from sunpy.physics.solar_rotation import mapsequence_solar_derotate

In [ ]:
derotated = mapsequence_solar_derotate(map_seq, layer_index=2, clip=False)

In [ ]:
for m in derotated:
    m.plot_settings['norm'].vmin = 0
    m.plot_settings['norm'].vmax = derotated[2].data.max()

In [ ]:
derotated.peek()

## `Map`

In [ ]:
aiamap = map_seq[2]

In [ ]:
aiamap.meta

In [ ]:
aiamap.observer_coordinate

## Plotting

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = plt.subplot(projection=aiamap)
aiamap.plot(ax)
aiamap.draw_grid()

Identifying a coronal hole
---
In the above AIA 193 image, there is a dark patch in the middle of the disc. This is a coronal hole, which is the source of the fastest solar wind.

This observation can be connected to in-situ measurements of the solar wind at 1 AU by looking for the fast solar wind stream that emmenates from this coronal hole. Because the solar wind takes a finite amount of time to propagate from the Sun to Earth, we first do an order of magnitude estimate of this delay.

Using astropy units
---
To calculate the propagation delay we can use the ``astropy.units`` module. This provides an extension of normal numbers and arrays, and allows units to be attached. All the unit mathematics is calculated automatically, avoiding the need to keep track of specific units.

In [ ]:
import astropy.constants as const
from sunpy.coordinates.ephemeris import get_earth

Assume the solar wind is relesed from the surface of the Sun, so the propagation distance is $D_{sun} - R_{sun}$

In [ ]:
d_sun = get_earth(aiamap.date)
d_sun

In [ ]:
d = (d_sun.radius - const.R_sun)

Take a typical fast solar wind speed of 500 km/s

In [ ]:
vsw = 500 * u.km / u.s
vsw

Calculate tne propagation time, and convert it to units of days

In [ ]:
t = (d / vsw).to(u.day)
t

In [ ]:
estimated_arrival = aiamap.date + t
estimated_arrival

Downloading and importing in-situ data
---
The ``heliopy.data`` module can be used to download and import a wide range of in situ datasets from various heliospheric missions. In this example we use data from OMNI, which provides measurements of the solar wind at the orbit of the Earth.

In [ ]:
from heliopy.data import omni

In [ ]:
starttime = aiamap.date - (1*u.year / 12)
endtime = aiamap.date + (1*u.year / 12)
data = omni.low(starttime.datetime, endtime.datetime)

The data is stored in the ``data`` object. We can print the available columns in this object:

In [ ]:
data

In [ ]:
data.meta

In [ ]:
for col in data.columns:
    print(col)

Plotting in-situ data
---
Matplotlib can be used to plot the downloaded data. In this example we plot the solar wind speed and the magnetic field clock angle, to see different polarity solar wind streams.

We also add a vertical line where the stream is fast stream is predicted to have arrived using the above back-of-the-envelope calculate. We can see that it lines up nicely with a fast solar wind stream that has speeds of 500 - 600 km/s.

In [ ]:
from astropy.visualization import quantity_support
quantity_support()

In [ ]:
fig, axs = plt.subplots(figsize=(9, 6), nrows=3, sharex=True)

ax = axs[0]
ax.plot(data.index, data.quantity('Plasma Flow Speed'), label='$v_{sw}$')
ax.axvline(estimated_arrival.datetime, color='k')

ax = axs[1]
ax.plot(data.index, data.quantity('|B|'), label='$|B|$')

ax = axs[2]
ax.plot(data.index, data.quantity('Proton Density'), label='$n_{p}$')

Improving figure formatting
---

In [ ]:
# Make the x-axis formatting nicer
fig.autofmt_xdate()
fig.subplots_adjust(hspace=0)

# Add a legend to each axes
for ax in axs:
    ax.legend()
    ax.axvline(estimated_arrival.datetime, color='k')

# Show the figure again
fig

Save a copy of the figure
---

In [ ]:
fig.savefig('tseries.pdf', bbox_inches='tight')